In [ ]:
import copy
import random
import collections
import warnings

import numpy as np

!pip install gymnasium
import gymnasium as gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as distributions
import torch.distributions as dist

from tqdm import tqdm
import matplotlib.pyplot as plt

!pip install omegaconf
from omegaconf import OmegaConf

from gym.wrappers import RecordVideo

ray install

In [ ]:
!pip install ray
import ray

# **Random seed fix**

In [40]:
import random

seed = 40
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
warnings.filterwarnings('ignore')

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mujoco install

In [ ]:
!pip show mujoco
!pip install --upgrade gymnasium[mujoco]

gym : half cheetah v4

In [285]:
env = gym.make('HalfCheetah-v4', exclude_current_positions_from_observation=False , render_mode = 'rgb_array')

In [286]:
def compute_log_prob(mean, log_std, action_raw):
    # Gaussian 로그 확률 계산
    std = log_std.exp()
    gaussian_log_prob = -0.5 * (((action_raw - mean) / std) ** 2 + 2 * log_std + torch.log(torch.tensor(2 * torch.pi)))
    gaussian_log_prob = gaussian_log_prob.sum(axis=-1)  # 차원 합산

    # Tanh 변환 보정 계산
    log_det_jacobian = torch.log(1 - torch.tanh(action_raw) ** 2 + 1e-6).sum(axis=-1)

    # 최종 로그 확률
    log_prob = gaussian_log_prob - log_det_jacobian
    return log_prob

In [287]:
@ray.remote
class ReplayBuffer():
    def __init__(self, config):
        self.config = config
        self.buffer = collections.deque(maxlen=self.config["buffer_limit"])

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
      mini_batch = random.sample(self.buffer, n)
      s_lst, a_lst, na_lst, r_lst, next_s_lst, done_mask_lst = [], [], [], [], [], []

      for transition in mini_batch:
        s, a, na, r, next_s, done = transition
        s_lst.append(s.tolist())
        a_lst.append(a.tolist())
        na_lst.append(na.tolist())
        r_lst.append([r])
        next_s_lst.append(next_s.tolist())
        done_mask = 0.0 if done else 1.0
        done_mask_lst.append([done_mask])

    # 반환값은 이미 텐서로 변환됨
      return (
        torch.Tensor(s_lst),
        torch.Tensor(a_lst),
        torch.Tensor(na_lst),
        torch.Tensor(r_lst),
        torch.Tensor(next_s_lst),
        torch.Tensor(done_mask_lst)
      )


    def size(self):
        return len(self.buffer)

    def remove_old_data(self, threshold):
        self.buffer = collections.deque(
            [transition for transition in self.buffer if transition[-1] > threshold],
            maxlen=self.config.buffer_limit
        )


In [288]:
def soft_update(target, source, tau):
    for target_param, source_param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(tau * source_param.data + (1.0 - tau) * target_param.data)

 35%|███▌      | 7/20 [08:23<19:18, 89.15s/it]


In [289]:
class Actor(nn.Module):
  def __init__(self,config):
    super().__init__()
    self.config=config
    self.tanh = nn.Tanh()
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax()
    self.dropout = nn.Dropout(0.2)

    self.actor_l1 = nn.Linear(config["state_dim"], config["hidden_dim"])
    self.actor_l2 = nn.Linear(config["hidden_dim"], config["hidden_dim"])
    self.actor_l3 = nn.Linear(config["hidden_dim"], config["hidden_dim"])

    self.actor_mean = nn.Linear(config["hidden_dim"] , config["action_dim"])
    self.actor_std = nn.Linear(config["hidden_dim"] , config["action_dim"])

  def forward(self, state):

    x = self.actor_l1(state)
    x = self.relu(x)
    x = self.actor_l2(x)
    x = self.relu(x)
    x = self.actor_l3(x)
    x = self.relu(x)

    mean_x = self.actor_mean(x)
    std_x = self.actor_std(x)
    std_x = torch.clamp(std_x, min= 1e-10 , max = 0.5)
    log_std_x = torch.log(std_x)

    normal = dist.Normal(mean_x, std_x)
    z = normal.rsample()  # reparameterization trick

    log_prob = compute_log_prob(mean_x, log_std_x, torch.Tensor(z))

    action = self.tanh(z)
    policy = log_prob

    return action , policy

In [290]:
class Critic(nn.Module):
  def __init__(self,config):
    super().__init__()
    self.data = []
    self.config=config
    self.tanh = nn.Tanh()
    self.relu = nn.ReLU()
    self.swish = nn.SiLU()
    self.dropout = nn.Dropout(0.1)

    self.critic_l1 = nn.Linear(config["state_dim"] + config["action_dim"], config["hidden_dim"])
    self.critic_l2 = nn.Linear(config["hidden_dim"], config["hidden_dim"])
    self.critic_l3 = nn.Linear(config["hidden_dim"], 1)

  def forward(self, input):
    x = self.critic_l1(input)
    x = self.relu(x)
    x = self.critic_l2(x)
    x = self.relu(x)
    x = self.critic_l3(x)
    return x

In [291]:
config = {
        'actor_class': Actor,
        'critic_class': Critic,
        'env_class': env,
        'lr_actor': 0.003,
        'lr_critic': 0.003,
        'num_episodes': 20,
        'batch_size': 256,
        'buffer_limit' : 3000000,
        'gamma': 0.99,
        'tau': 0.01,
        'buffer_size': 10000,
        'num_workers': 8,
        'state_dim' : env.observation_space.shape[0],
        'action_dim' : int(env.action_space.shape[0]),
        'hidden_dim' : 256,
    }

In [292]:
actor = config['actor_class'](config)

In [293]:
@ray.remote
def train_loop_per_worker(config , actor):
  critic1 = config['critic_class'](config)
  critic2 = config['critic_class'](config)
  target_critic = config['critic_class'](config)

  actor_optimizer = optim.Adam(actor.parameters(), lr=config['lr_actor'])
  critic1_optimizer = optim.Adam(critic1.parameters(), lr=config['lr_critic'])
  critic2_optimizer = optim.Adam(critic2.parameters(), lr=config['lr_critic'])

  memory = ReplayBuffer.remote(config)

  epi_rews = []

  # 학습 시작
  for n_epi in tqdm(range(config['num_episodes'])):
    state, _ = env.reset()
    terminated, truncated = False, False
    epi_rew = 0

    while not (terminated or truncated):
      state = torch.Tensor(state)
      action, policy = actor(state)

      # 환경에서 다음 상태 및 보상 얻기
      next_state, reward, terminated, truncated, _ = env.step(action.detach().numpy())
      next_state = torch.Tensor(next_state)
      next_action, _ = actor(next_state)

      # 메모리에 저장
      ray.get(memory.put.remote([state, action, next_action, reward, next_state, terminated or truncated]))

      # 메모리 샘플링 및 업데이트
      if ray.get(memory.size.remote()) > 2000 :
        for _ in range(2):
          samples = ray.get(memory.sample.remote(config['batch_size']))
          samples = (samples[0], samples[1], samples[2], samples[3], samples[4], samples[5])

          states, actions, next_actions, rewards, next_states, dones = samples

          critic_inputs = torch.cat([states, actions], axis=1)
          next_critic_inputs = torch.cat([next_states, next_actions], axis=1)

          # Target critic 업데이트
          binary = torch.sum(critic1(next_critic_inputs)) > torch.sum(critic2(next_critic_inputs))
          if binary:
            soft_update(target_critic, critic1, config['tau'])
          else:
            soft_update(target_critic, critic2, config['tau'])

          # 손실 함수 계산
          actor_loss = -target_critic(critic_inputs).mean()
          critic1_loss = nn.MSELoss()(critic1(critic_inputs),
                                                rewards + config['gamma'] * (1 - dones) * target_critic(next_critic_inputs))
          critic2_loss = nn.MSELoss()(critic2(critic_inputs),
                                                rewards + config['gamma'] * (1 - dones) * target_critic(next_critic_inputs))

          # 역전파 및 최적화
          actor_optimizer.zero_grad()
          critic1_optimizer.zero_grad()
          critic2_optimizer.zero_grad()

          actor_loss.backward()
          critic1_loss.backward()
          critic2_loss.backward()

          actor_optimizer.step()
          critic1_optimizer.step()
          critic2_optimizer.step()

          epi_rew += reward
          state = next_state

    epi_rews.append(epi_rew)

  return epi_rews

In [296]:
# Ray의 병렬 작업 실행
def train_distributed(config , actor):
    num_workers = config['num_workers']
    results = ray.get([train_loop_per_worker.remote(config , actor) for _ in range(num_workers)])
    return results

In [ ]:
ray.init()

In [297]:
results , actor = train_distributed(config , actor)
print("Training results:", results)

(raylet) WARNING: 56 PYTHON worker processes have been started on node: 7be1e42d29b3e0caf95b1009b422d389912cc10cd519285bb8327d4f with address: 172.28.0.12. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).


 31%|███       | 31/100 [25:27<1:42:58, 89.55s/it] [repeated 7x across cluster]


KeyboardInterrupt: 

In [ ]:
ray.shutdown()

# **visualization**

In [298]:
import os
os.environ['MUJOCO_GL']='egl'
env = gym.wrappers.RecordVideo(env, video_folder='./videos')

 31%|███       | 31/100 [25:40<1:44:04, 90.50s/it]


In [299]:
state,_ = env.reset()
terminated, truncated = False, False
for i in range(1):
    while not (terminated or truncated):
      state = torch.Tensor(state)
      action , policy = actor(state)
      action = action.detach().numpy()
      next_state, reward, terminated, truncated, _ = env.step(action)

      state = next_state

env.close()

 45%|████▌     | 9/20 [13:49<23:41, 129.24s/it] [repeated 2x across cluster]


In [300]:
from IPython.display import Video
Video("./videos/rl-video-episode-0.mp4", embed=True)